In [3]:
from importlib import reload
import pandas as pd
import numpy as np
from datetime import datetime
import datatools
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import pvlib
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import pvanalytics
import seaborn

%matplotlib inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
mpl.rcParams['font.weight']='bold'
mpl.rcParams['axes.titlesize']=22

In [2]:
def sys_met_data(mettablename, systablename, start, end):
    engine = datatools.database.create_mss_engine()
    
    met_sql = f"select * from {mettablename} where TmStamp between '{start}' and '{end}';"
    met = pd.read_sql(met_sql, engine, index_col='TmStamp')
    met.index = pd.DatetimeIndex(met.index)
    met.index = met.index.tz_localize('MST') 
    
    sys_sql = f"select * from {systablename} where TmStamp between '{start}' and '{end}';"
    sys = pd.read_sql(sys_sql, engine, index_col='TmStamp')
    sys.index = pd.DatetimeIndex(sys.index)
    sys.index = sys.index.tz_localize('MST')

    df = pd.merge(met,sys,how='inner', left_index=True, right_index=True)
    
    return df

In [ ]:
def visual_dots(sys, mask, ylim_lower, ylim_upper):

    masked_sys = sys[~mask]
    plt.scatter(sys.index, sys, c='#add8e6',s=6, label='data', alpha=0.4)
    plt.scatter(masked_sys.index, masked_sys, c='#da9b86', label='mask')
    plt.xlabel(sys.index.name)
    plt.ylabel(sys.name)
    plt.ylim(ylim_lower, ylim_upper)
    #plt.xlim(xlim_lower, xlim_upper)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.show()
    return

In [ ]:
 def simple_filter(sys, variable, lower=None, upper=None, apply=False):

    variable = sys[variable]

# Physical Limits  (note: mask = True for data in acceptable ranges)


    if lower==None:
        mask_low = pd.Series(dtype='float64')
        if upper==None:
            return print('Please specify upper and/or lower limits')
        else:
            mask_high = variable<=upper
            mask = mask_high
    else:
        mask_low = variable >= lower
        if upper==None:
            mask_high = pd.Series(dtype='float64')
            mask = mask_low
        else:
            mask_high = variable<= upper
            mask = mask_low & mask_high
        
### Should add a check if the values are NaNs?  
    mask.name = f'mask_{variable.name}'

    if apply==True:
        sys = sys.where(mask, np.nan)
        return sys
    else:
        return mask